# Introducción a OpenAI GYM
**Ingeniería Electrónica**

**Inteligencia Artificial**

**19/06/2021**

Para implementar un algoritmo de Q-learning, usaremos el framework de OpenAI Gym, el cual es compatible con TensorFlow para desarrollar y comparar algoritmos de RL.

OpenAI Gym consta de dos partes principales:
* **La librería de código abierto Gym**: una colección de problemas y entornos que se pueden usar para probar los algoritmos de aprendizaje reforzado. Todos estos entornos tienen una interfaz compartida, lo que le permite escribir algoritmos RL.
* **El servicio OpenAI Gym**: un sitio y API que permite a las personas comparar significativamente el rendimiento de sus agentes entrenados.

Ver más referencias en https://gym.openai.com



Para instalar Gym, use el instalador pip:

In [ ]:
! pip install gym
#! conda install -c conda-forge gym

Una vez instalado, se pueden revisar los entornos de Gym de la siguiente manera:

In [ ]:
from gym import envs
envs.registry.all()

Cada `EnvSpec` define una tarea para resolver, por ejemplo, la representación de `FrozenLake-v0` se da en la siguiente figura. El agente controla el movimiento de un personaje en un mundo de cuadrícula 4x4 (ver la siguiente figura). Algunas baldosas de la cuadrícula son transitables, y otras conducen al agente a caer al agua. Además, la dirección de movimiento del agente es incierta, y solo depende parcialmente de la dirección elegida. El agente es recompensado por encontrar una ruta transitable a una casilla de meta:

<img src="cuadricula.png">

La superficie mostrada anteriormente se describe utilizando una cuadrícula, como la siguiente:

SFFF (S: punto de partida, seguro)

FHFH (F: superficie congelada, segura)

FFFH (H: agujero, cae a tu perdición)

HFFG (G: meta, donde se encuentra el objetivo)


El *episodio* termina cuando alcanzamos la meta o caemos en un hoyo. Recibimos una recompensa de uno por alcanzar la meta y cero en caso contrario.

## Problema de implementación de FrozenLake-v0
Aquí presentamos una implementación básica de Q-learning para el problema FrozenLake-v0.

Importar las siguientes dos librerías básicas:

In [ ]:
import gym
import numpy as np
import random

Luego, cargamos el entorno `FrozenLake-v0`:

In [ ]:
random.seed(2021)
environment = gym.make('FrozenLake-v0')

Luego, construimos la tabla Q-learning; tiene las dimensiones *SxA*, donde `S` es la dimensión del espacio de observación, mientras que `A` es la dimensión del espacio de acción:

In [ ]:
S = environment.observation_space.n
A = environment.action_space.n

El entorno FrozenLake proporciona un estado para cada bloque y cuatro acciones (es decir, las cuatro direcciones de movimiento), lo que nos proporciona una tabla de valores Q de 16x4 para inicializar:

In [ ]:
Q = np.zeros([S,A])

Luego, definimos parámetros para la regla de entrenamiento y el factor de descuento. Establecemos el número total de episodios (pruebas). Luego, inicializamos `rList`, donde agregaremos la recompensa acumulada para evaluar la puntuación del algoritmo:

In [16]:
lr = .85
alpha = .99
gamma = .99
num_episodes = 2000
rList = []

Finalmente, comenzamos el ciclo de Q-learning:

In [17]:
for i in range(num_episodes):
# Restablecer el entorno y obtener la primera observación nueva
    s = environment.reset()
    rAll = 0
    d = False
    j = 0

  # El algoritmo de aprendizaje Q-Table
    while j < 99:
        j+=1

   # Elegir una acción escogiendo (con ruido) de la tabla Q
        a=np.argmax(Q[s,:]+ \
                  np.random.randn(1,environment.action_space.n)*(1./(i+1)))

  # Obtener un nuevo estado y una recompensa del entorno
        s1,r,d,_ = environment.step(a)

        # Actualizar Q-Table con nuevos conocimientos
        Q[s,a] = Q[s,a] + lr*(r + gamma *np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break

    rList.append(rAll)

In [ ]:
print("Puntuación en el tiempo: " + str(sum(rList)/num_episodes))
print("Valores finales de la tabla Q")
print(Q)

La recompensa promedio es de aproximadamente 0.47 en 100 pruebas consecutivas. Técnicamente, no lo resolvimos. De hecho, `FrozenLake-v0` define la resolución como obtener una recompensa promedio de 0.78 en más de 100 pruebas consecutivas; podrían mejorar este resultado ajustando los parámetros de configuración.

## Q-learning con TensorFlow
En el ejemplo anterior, vimos cómo es relativamente simple, utilizando una cuadrícula de 16x4, actualizar la tabla Q en cada paso del proceso de aprendizaje. Es fácil imaginar que el uso de esta tabla puede servir para problemas simples, pero en problemas del mundo real, necesitamos un mecanismo más sofisticado para actualizar el estado del sistema. Este es el punto donde interviene el Deep Learning. Las redes neuronales son excepcionalmente buenas para generar buenas características para datos altamente estructurados.

Veremos cómo usar una función Q con una red neuronal, que toma el estado y la acción como entrada y emite el valor Q correspondiente. Para hacer eso, construiremos una red de una capa que tome el estado, codificado en un vector [1x16], que aprenda el mejor movimiento (acción), mapeando las acciones posibles en un vector de longitud cuatro.

Una aplicación de redes Q profundas (**_deep Q-networks_**) ha tenido éxito en jugar algunos juegos de Atari 2600 a niveles humanos expertos. Los resultados preliminares se presentaron en 2014, con un artículo publicado en febrero de 2015, en *Nature*.

A continuación, se describe una implementación basada en TensorFlow de una red neuronal Q-learning para el problema `FrozenLake-v0`.

Importar las librerías necesarias:

In [19]:
import gym
import numpy as np
import random
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt

In [20]:

# Definir el entorno FrozenLake
env = gym.make('FrozenLake-v0')

# Configurar las variables de TensorFlow
tf.reset_default_graph()
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)

# definir las funciones de pérdida y optimización
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

#initilizar las variables
init = tf.global_variables_initializer()

# preparar los parámetros de q-learning
gamma = .99
e = 0.1
num_episodes = 6000
jList = []
rList = []

# Ejecutar la sesión
with tf.Session() as sess:
    sess.run(init)
# Iniciar el procedimiento de Q-learning
    for i in range(num_episodes):
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        while j < 99:
            j+=1
            a,allQ = sess.run([predict,Qout],\
                              feed_dict=\
                              {inputs1:np.identity(16)[s:s+1]})

            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            s1,r,d,_ = env.step(a[0])
            Q1 = sess.run(Qout,feed_dict=\
                          {inputs1:np.identity(16)[s1:s1+1]})
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + gamma *maxQ1
            _,W1 = sess.run([updateModel,W],\
                            feed_dict=\
                           {inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
# acumular la recompensa total
            rAll += r
            s = s1
            if d == True:
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
# imprimir los resultados
    print("Porcentaje de episodios exitosos: " + str(sum(rList)/num_episodes) + "%")

Porcentaje de episodios exitosos: 0.584%


Todavía se podría mejorar ajustando los parámetros de la red.

## Otros ejemplos con OpenAI
### implementación de CartPole-v1

Un tubo está unido por una articulación no accionada a un carro, que se mueve a lo largo de una pista sin fricción. El sistema se controla aplicando una fuerza de +1 o -1 al carro. El péndulo comienza en posición vertical, y el objetivo es evitar que se caiga. Se proporciona una recompensa de +1 por cada paso de tiempo que el tubo permanece en posición vertical. El episodio termina cuando el tubo está a más de 15 grados de la vertical, o el si el carro se mueve a más de 2.4 unidades del centro.

In [ ]:
import gym
env = gym.make("CartPole-v1")
observation = env.reset()
for _ in range(1000):
    env.render()
action = env.action_space.sample()  # el agente toma acciones aleatorias
observation, reward, done, info = env.step(action)
if done:
    observation = env.reset()
env.close()

La segunda línea crea un entorno `CartPole-v1`.

La tercera línea inicializa los parámetros de estado.

La quinta línea muestra el juego.

La sexta línea genera una acción con una política que es "aleatoria".

En la séptima línea, se toma la acción y el entorno ofrece cuatro resultados:
* **Observation**: Parámetros del estado del juego. Diferentes juegos devuelven diferentes parámetros. En CartPole, hay cuatro en total. El segundo parámetro es el ángulo del poste.
* **Reward**: la puntuación que obtienes después de realizar esta acción.
* **Done**: Juego terminado o no terminado.
* **Info**: información adicional de depuración. Podría estar haciendo trampa.

La octava línea significa si el juego está terminado; reiniciarlo.

La política se puede programar de la forma que se desee. Se puede basar en reglas if-else o en una red neuronal. Aquí hay una pequeña demostración simple, con la política más simple para el juego CartPole.

In [ ]:
import gym
def policy(observation):
    angle = observation[2]
    if angle < 0:
        return 0
    else:
        return 1
env = gym.make("CartPole-v1")
observation = env.reset()
for _ in range(1000):
    env.render()
    action = policy(observation)
    observation, reward, done, info = env.step(action)
    if done:
        observation = env.reset()
env.close()